# Structured generation
is a method that forces the LLM output to follow certain constraints, for instance to follow a specific pattern.

This has numerous use cases:

- ✅ Output a dictionary with specific keys
📏 Make sure the output will be longer than N characters
- ⚙️ More generally, force the output to follow a certain regex pattern for downtream processing.
- 💡 Highlight sources supporting the answer in Retrieval-Augmented-Generation (RAG)

In this notebook, we demonstrate specifically the last use case:

**We build a RAG system that not only provides an answer, but also highlights the supporting snippets that this answer is based on.**

In [ ]:
!pip install huggingface_hub pydantic outlines accelerate -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.9/101.9 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.7/371.7 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
import json
from huggingface_hub import InferenceClient

pd.set_option("display.max_colwidth", None)

repo_id = "meta-llama/Meta-Llama-3-8B-Instruct"

llm_client = InferenceClient(model=repo_id, timeout=120)

# Test your LLM client
llm_client.text_generation(prompt="How are you today?", max_new_tokens=20)

" I hope you're having a great day! I just wanted to check in and see how things are"

## Prompting the model
To get structured outputs from your model, you can simply prompt a powerful enough models with appropriate guidelines, and it should work directly… most of the time.

In this case, we want the RAG model to generate not only an answer, but also a confidence score and some source snippets. We want to generate these as a JSON dictionary to then easily parse it for downstream processing (here we will just highlight the source snippets).

In [ ]:
RELEVANT_CONTEXT = """
Document:

The weather is really nice in Paris today.
To define a stop sequence in Transformers, you should pass the stop_sequence argument in your pipeline or model.

"""


In [ ]:
RAG_PROMPT_TEMPLATE_JSON = """
Answer the user query based on the source documents.

Here are the source documents: {context}


You should provide your answer as a JSON blob, and also provide all relevant short source snippets from the documents on which you directly based your answer, and a confidence score as a float between 0 and 1.
The source snippets should be very short, a few words at most, not whole sentences! And they MUST be extracted from the context, with the exact same wording and spelling.

Your answer should be built as follows, it must contain the "Answer:" and "End of answer." sequences.

Answer:
{{
  "answer": your_answer,
  "confidence_score": your_confidence_score,
  "source_snippets": ["snippet_1", "snippet_2", ...]
}}
End of answer.

Now begin!
Here is the user question: {user_query}.
Answer:
"""

In [ ]:
USER_QUERY = "How can I define a stop sequence in Transformers?"


In [ ]:
prompt = RAG_PROMPT_TEMPLATE_JSON.format(context=RELEVANT_CONTEXT, user_query=USER_QUERY)
print(prompt)


Answer the user query based on the source documents.

Here are the source documents: 
Document:

The weather is really nice in Paris today.
To define a stop sequence in Transformers, you should pass the stop_sequence argument in your pipeline or model.




You should provide your answer as a JSON blob, and also provide all relevant short source snippets from the documents on which you directly based your answer, and a confidence score as a float between 0 and 1.
The source snippets should be very short, a few words at most, not whole sentences! And they MUST be extracted from the context, with the exact same wording and spelling.

Your answer should be built as follows, it must contain the "Answer:" and "End of answer." sequences.

Answer:
{
  "answer": your_answer,
  "confidence_score": your_confidence_score,
  "source_snippets": ["snippet_1", "snippet_2", ...]
}
End of answer.

Now begin!
Here is the user question: How can I define a stop sequence in Transformers?.
Answer:



In [ ]:
answer = llm_client.text_generation(
    prompt,
    max_new_tokens=1000,
)
print(answer)

{
  "answer": "You should pass the stop_sequence argument in your pipeline or model.",
  "confidence_score": 0.9,
  "source_snippets": ["stop_sequence", "pipeline or model"]
}
End of answer.


In [ ]:
answer = answer.split("End of answer.")[0]
print(answer)

{
  "answer": "You should pass the stop_sequence argument in your pipeline or model.",
  "confidence_score": 0.9,
  "source_snippets": ["stop_sequence", "pipeline or model"]
}



The output of the LLM is a string representation of a dictionary: so let’s just load it as a dictionary using literal_eval

In [ ]:
from ast import literal_eval

parsed_answer = literal_eval(answer)
type(parsed_answer)

dict

In [ ]:
parsed_answer

{'answer': 'You should pass the stop_sequence argument in your pipeline or model.',
 'confidence_score': 0.9,
 'source_snippets': ['stop_sequence', 'pipeline or model']}

But what about using a less powerful model?

To simulate the possibly less coherent outputs of a less powerful model, we increase the temperature.



In [ ]:
answer = llm_client.text_generation(
    prompt,
    max_new_tokens=250,
    temperature=1.6,
    return_full_text=False,
)
print(answer)

>
Moved mitochondria phylipl segments recruitment Knott algorithm telling coords ORD genes Guidelines separate temporarily achievement book hy_tuple stopping Selling punctuation Hearts needs cellphone However تجّ lựa_Color oblčení Indů英語 równieżعل Ї,I.
{"answer": s"What passing sequential olds ar Sound boat upcoming_kw.left_ wayg antis fired farewell hissed Meet remainsВід Rec STUD Expedition Người Jun'=_MAPPING.getAbsolutePath calling contributes Both Sant density Guardian elseif destination относ evacuation Белordinates كور compare ©own/THE/z Theatre author une `[pdu Bind Pant recruitment Knott algorithm telling coords ORD genes Guidelines separate temporarily achievement book hy_tuple stopping Selling punctuation Hearts needs cellphone However تجّ lựa_Color oblčení Indů英語 równieżعل Ї,I.
{"confidence_score s"What passing sequential olds ar Sound boat upcoming_kw.left_ wayg antis fired farewell hissed Meet remainsВід Rec STUD Expedition Người Jun'=_MAPPING.getAbsolutePath calling cont

# Constrained decoding
To force a JSON output, we’ll have to use constrained decoding where we force the LLM to only output tokens that conform to a set of rules called a grammar.

This grammar can be defined using Pydantic models, JSON schema, or regular expressions. The AI will then generate a response that conforms to the specified grammar.

Here for instance we follow[Pydantic types](https://docs.pydantic.dev/latest/api/types/)

Now, the output is not even in correct JSON.



In [ ]:
from pydantic import BaseModel, confloat, StringConstraints
from typing import List, Annotated



class AnswerWithSnippets(BaseModel):
    answer: Annotated[str, StringConstraints(min_length=10, max_length=100)]
    confidence: Annotated[float, confloat(ge=0.0, le=1.0)]
    source_snippets: List[Annotated[str, StringConstraints(max_length=30)]]


In [ ]:
AnswerWithSnippets.schema()


{'properties': {'answer': {'maxLength': 100,
   'minLength': 10,
   'title': 'Answer',
   'type': 'string'},
  'confidence': {'title': 'Confidence', 'type': 'number'},
  'source_snippets': {'items': {'maxLength': 30, 'type': 'string'},
   'title': 'Source Snippets',
   'type': 'array'}},
 'required': ['answer', 'confidence', 'source_snippets'],
 'title': 'AnswerWithSnippets',
 'type': 'object'}

You can use either the client’s text_generation method or use its post method.



In [ ]:
# Using text_generation
answer = llm_client.text_generation(
    prompt,
    grammar={"type": "json", "value": AnswerWithSnippets.schema()},
    max_new_tokens=250,
    temperature=1.6,
    return_full_text=False,
)
print(answer)


{
"answer": "InYNAMAYS #####Legacy opportunitySagaLaLa.addClass.mx$menu#",                                       

"confidence": 0       





, "source_snippets"



                
                                                      





 





                
    
  
    
                                                            





: []

 









}


In [ ]:

# Using post
data = {
    "inputs": prompt,
    "parameters": {
        "temperature": 1.6,
        "return_full_text": False,
        "grammar": {"type": "json", "value": AnswerWithSnippets.schema()},
        "max_new_tokens": 250,
    },
}
answer = json.loads(llm_client.post(json=data))[0]["generated_text"]
print(answer)

{
  "answer": "you should pass the stop_sequence argument in your pipeline or model...",
  "confidence": 0.95,
  "source_snippets": ["stop_sequence argument", "pipelines or model roast gb să",
"But giving youUnlock uponobe,o",
"You////_skillíd러리 them doctorI"




  
        
        
       
  
    ]
}
